In [1]:
import tkinter as tk
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import time
import os
from tkinter.ttk import Progressbar
from pygame import mixer
from PIL import ImageTk, Image
from tkinter import PhotoImage
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk) 
from matplotlib.figure import Figure
from scipy.stats import poisson
from scipy.stats import kurtosis
from scipy.stats import skew
from tkinter import filedialog 

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def detector_simulator(lraw_counts, ang_solido, laceitancia_geom, leficiencia_detector):
    total_solido = round((lraw_counts*laceitancia_geom*leficiencia_detector), 2)
    return total_solido

In [3]:
def fonte_de_radiacao(lmi, lq_medidas, lj, ltempo_medida):
    data_poisson = poisson.rvs(mu = lmi, size = lq_medidas)
    return(data_poisson[int(lj)]*ltempo_medida)

In [4]:
def angulo_solido(lR, lh):
    global ang_solido
    r = np.sqrt(lh**2 + lR**2)
    sen_teta1 = lR/r
    teta = np.arcsin(sen_teta1)
    ang_solido = round(2*np.pi*(1-np.cos(teta)),2)
    print('Ângulo sólido =', angulo_solido, 'rad')
    return ang_solido

In [13]:
h = 0
j = 0
music = 0
musicon = False
janela_aberta = False
grafico_aberto = False
newWindow = 0
ativ = 0
num_medidas = 0
tempo_medida = 0
voltagem = 0
ang_solido = 0
list_total_medida = []
list_total_solido = []
main_line = 0
#Função para movimentar o detector
def arrastarCursor(event):
    global main_line
    if dentro(event.x, event.y) and event.y > 50 and event.y < 665:
        global h
        canvas.coords(detector, posicao_1[0], event.y-25, posicao_2[0], event.y+25)
        c = canvas.coords(detector)
        h = 689-c[3]
        if main_line != 0:
            canvas.delete(main_line)
        createLines()
        disttxt.delete('1.0', tk.END)
        disttxt.insert('1.0', h)
        
def detectorPos(event):
    global h
    nova = disttxt.get(1.0, 'end-1c')
    h = float(nova)
    canvas.coords(detector, posicao_1[0], 689-h-50, posicao_2[0], 689-h)
#Função que detecta se o mouse esta dentro da área do detector
def dentro(x, y):
    c = canvas.coords(detector)
    if x > c[0] and x < c[2] and y < c[3] and y > c[1]:
        return True
def create_circle(x, y, r, canvas): #center coordinates, radius
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    return canvas.create_oval(x0, y0, x1, y1, fill = 'black')
#Função para fechar a segunda janela
def fechaJanela():
    global janela_aberta
    global newWindow
    janela_aberta = False
    newWindow.destroy()
#Função para criar as caixas de texto 
def inputText():
    global ativ
    global num_medidas
    global tempo_medida
    global voltagem
    #ativ = float(ativtxt.get(1.0, 'end-1c'))
    #num_medidas = int(num_medidastxt.get(1.0, 'end-1c'))
    #tempo_medida = float(tempo_medidatxt.get(1.0, 'end-1c'))
    #voltagem = float(voltagemtxt.get(1.0, 'end-1c'))
    ativ = 82
    num_medidas = 5
    tempo_medida = 1
    voltagem = 950
def audio():
    global num_medidas
    global music
    global musicon
    if music < (num_medidas-1) and musicon == False:
        mixer.init()
        mixer.music.load('Som.mp3')
        mixer.music.play(loops=0)
        musicon = True
        music += 1
    elif music < (num_medidas-1) and musicon == True:
        mixer.music.queue('Som.mp3')
        music += 1
    else:
        mixer.music.stop()
        musicon = False
        music = 0
def salvarDados():
    filename = filedialog.asksaveasfilename(initialdir = "/", filetypes = (("Export files", "*.csv*"),))
    filename = filename + '.csv'
    df = pd.DataFrame(list_total_solido)
    df.to_csv(filename, index = False)
def createLines():
    global main_line
    main_line = canvas.create_line(275, 680, 275, 690-h, dash = (10,2))
    canvas.create_line(295, 700, 600, 700, dash = (10,2))
    canvas.create_line(255, 700, 0, 700, dash = (10,2))
    #canvas.create_line(275, 700, 600, 700, dash = (10,2))
    #canvas.create_line(275, 700, 600, 700, dash = (10,2))
    #canvas.create_line(275, 700, 600, 700, dash = (10,2))
    #canvas.create_line(275, 700, 600, 700, dash = (10,2))
def clickGraph():
    global list_total_solido
    global grafico_aberto
    global tempo_medida
    global num_medidas
    if grafico_aberto:
        plt.close('all')
        grafico_aberto = False
        clickGraph()
    else:
        janelaHist = tk.Toplevel(root)
        janelaHist.geometry("500x500")
        janelaHist.title("Histograma")
        def update_frequency(number):
            plt.hist(list_total_solido[0:int(number)])
            canvasHist.draw()
            canvasHist.get_tk_widget().pack()
        plt.hist(list_total_solido[0:1])
        fig = plt.gcf()
        canvasHist = FigureCanvasTkAgg(fig, master = janelaHist)
        toolbar = NavigationToolbar2Tk(canvasHist, janelaHist)
        toolbar.update()
        canvasHist.get_tk_widget().pack(side=tk.TOP)
        slider_update = tk.Scale(janelaHist, from_=1, to=num_medidas, orient=tk.HORIZONTAL,
                                 command=update_frequency, label="Medidas")
        slider_update.pack(side=tk.BOTTOM)
        grafico_aberto = True

def clickPronto():
    global janela_aberta
    global newWindow
    global j
    global ativ
    global num_medidas
    global tempo_medida
    global voltagem
    global list_total_solido
    global list_total_medida
    global h
    if not janela_aberta:
        #Definindo a nova janela
        newWindow = tk.Toplevel(root)
        newWindow.title("Resultados")
        newWindow.geometry("500x500")
        inputText()
        #Rodar funções externas
        list_total_solido = []
        list_total_medida = []
        ang_solido = angulo_solido(125, h)
        aceitancia_geom = ang_solido/(4*np.pi)
        eficiencia_detector = 0.01*((2/25)*voltagem+24)/100
        while j < num_medidas:
            raw_counts = fonte_de_radiacao(ativ, num_medidas, j, tempo_medida)
            counts = detector_simulator(raw_counts, ang_solido, aceitancia_geom, eficiencia_detector)
            list_total_medida.append(raw_counts)
            list_total_solido.append(counts)
            audio()
            time.sleep(tempo_medida)
            j+=1
        media_total = round(sum(list_total_medida)/len(list_total_medida), 2)
        media_solido = sum(list_total_solido)/len(list_total_solido)
        desvio_padrao_total = round(np.std(list_total_medida), 2)
        desvio_padrao_solido = np.std(list_total_solido)
        #skewness_total = skew(list_total_medida)
        skewness_medida = skew(list_total_solido)
        #kurtosis_total = kurtosis(list_total_medida)
        kurtosis_medida = kurtosis(list_total_solido)
        #print(list_total_solido)
        #print('Média de decaimentos =', media_total, '\nMédia de decaimentos detectados =', media_solido)
        #print('Desvio padrão total =', desvio_padrao_total, '\nDesvio padrão medido =', desvio_padrao_solido)
        #print('Skewness total =', skewness_total, '\nSkewness das medidas =', skewness_medida)
        #print('Kurtosis total =', kurtosis_total, '\nKurtosis das medidas =', kurtosis_medida)
        tk.Label(newWindow, text = "Distância: " + str(h), font = 32).pack()
        tk.Label(newWindow, text = "Ângulo sólido: " + str(ang_solido), font = 32).pack()
        #tk.Label(newWindow, text = "Atividade da fonte: " + str(ativ)).pack()
        tk.Label(newWindow, text = "Média de decaimentos: " + str(media_total), font = 32).pack()
        tk.Label(newWindow, text = "Desvio padrão: " + str(desvio_padrao_total), font = 32).pack()
        newWindow.protocol('WM_DELETE_WINDOW', fechaJanela)
        j = 0
        janela_aberta = True
    else:
        fechaJanela()
        clickPronto()
        
root = tk.Tk()
root.configure(bg = 'grey')
root.state('zoomed')
root.geometry("500x500")
root.title('Detector')

#Objetos
Botao = tk.Button(root, text = "Pronto!", bg = 'red', fg = "black", font = 32, command = clickPronto)
#Botao2 = tk.Button(root, text = "Histograma", bg = 'blue', fg = 'black', font = 32, command = clickGraph)
txt0 = tk.Label(root, text = "Distância entre a fonte e o detector em centímetros:", bg = 'grey', font = 32)
txt1 = tk.Label(root, text = "Insira a atividade da fonte em Curie:", bg = 'grey', font = 32)
txt2 = tk.Label(root, text = "Insira o número de medidas desejadas:", bg = 'grey', font = 32)
txt3 = tk.Label(root, text = "Insira o tempo de cada medida em segundos:", bg = 'grey', font = 32)
txt4 = tk.Label(root, text = "Insira a voltagem no detector:", bg = 'grey', font = 32)

#Criar desenhos
canvas = tk.Canvas(root, bg = 'grey', width = 550, height = 820)

posicao_1 = [150, 640]
posicao_2 = [400, 690]

detector = canvas.create_rectangle(posicao_1[0], posicao_1[1], posicao_2[0], posicao_2[1], fill='red')
fonte = create_circle(275, 700, 10, canvas)

disttxt = tk.Text(root, height = 1.3, width = 15, font = 15)
disttxt.insert('1.0', h)
root.bind('<Key>', detectorPos)
canvas.bind('<Button-1>', createLines)
canvas.bind('<Button-1>', deleteLines)
#disttxt.bind('<Control_L>', detectorPos())
ativtxt = tk.Text(root, height = 1.3, width = 15, font = 15)
num_medidastxt = tk.Text(root, height = 1.3, width = 15, font = 15)
tempo_medidatxt = tk.Text(root, height = 1.3, width = 15, font = 15)
voltagemtxt = tk.Text(root, height = 1.3, width = 15, font = 15)

canvas.bind('<B1-Motion>', arrastarCursor)
#Barra de Menu
barraDeMenus = tk.Menu(root)
menuOpcoes = tk.Menu(barraDeMenus, tearoff = 0)
menuOpcoes.add_command(label = 'Histograma', command = clickGraph)
menuOpcoes.add_command(label = 'Salvar medidas de decaimento', command = salvarDados)
barraDeMenus.add_cascade(label = 'Arquivo', menu = menuOpcoes)
root.config(menu = barraDeMenus)
#Posição de desenhos
canvas.place(x = 200, y = 0)
#Posição dos botões
Botao.place(x = 1350, y = 700, height = 50, width = 150)
#Posição dos textos
txt0.place(x = 800, y = 70)
txt1.place(x = 800, y = 150)
txt2.place(x = 800, y = 210)
txt3.place(x = 800, y = 270)
txt4.place(x = 800, y = 330)
#Posição das caixas
disttxt.place(x = 800, y = 100)
ativtxt.place(x = 800, y = 180)
num_medidastxt.place(x = 800, y = 240)
tempo_medidatxt.place(x = 800, y = 300)
voltagemtxt.place(x = 800, y = 360)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\fisst\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
TypeError: deleteLines() takes 0 positional arguments but 1 was given
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\fisst\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
TypeError: deleteLines() takes 0 positional arguments but 1 was given
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\fisst\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
TypeError: deleteLines() takes 0 positional arguments but 1 was given
